# Import Libraries

In [1]:
import os
import numpy as np
import bs4
import time
import requests
import lxml
import urllib
import urllib.request

from pprint import pprint
from bs4 import BeautifulSoup
from tqdm import tqdm
from getpass import getpass
from urllib.parse import urljoin, urlparse

# Parse all websites of the Manga

## colored

In [2]:
html_text = requests.get('https://ww9.readonepiece.com/manga/one-piece-digital-colored-comics/', 'r').text
soup = BeautifulSoup(html_text, 'lxml')

cl_chapters = soup.find_all('div', class_ = 'bg-bg-secondary p-3 rounded mb-3 shadow')
cl_chapter_site = []
for chapter in cl_chapters:
    site = chapter.find('a', class_ = 'uppercase text-xs bg-text-success text-white font-bold py-1 px-2 rounded').attrs['href']
    cl_chapter_site.append(site)

# cl_chapter_site.sort()
pprint(len(cl_chapter_site))
pprint(cl_chapter_site[0])
pprint(cl_chapter_site[-1])

988
'https://ww9.readonepiece.com/chapter/one-piece-digital-colored-comics-chapter-1048'
'https://ww9.readonepiece.com/chapter/one-piece-digital-colored-comics-chapter-001'


## grayscale

In [3]:
html_text = requests.get('https://ww9.readonepiece.com/manga/one-piece/', 'r').text
soup = BeautifulSoup(html_text, 'lxml')

bw_chapters = soup.find_all('div', class_ = 'bg-bg-secondary p-3 rounded mb-3 shadow')
bw_chapter_site = []
for chapter in bw_chapters:
    site = chapter.find('a', class_ = 'uppercase text-xs bg-text-success text-white font-bold py-1 px-2 rounded').attrs['href']
    bw_chapter_site.append(site)

# bw_chapter_site.sort()
pprint(len(bw_chapter_site))
pprint(bw_chapter_site[0])
pprint(bw_chapter_site[-1])

1060
'https://ww9.readonepiece.com/chapter/one-piece-chapter-1050'
'https://ww9.readonepiece.com/chapter/one-piece-chapter-001'


In [4]:
cl_chapters = []
bw_chapters = []

for i in tqdm(range(len(cl_chapter_site)), 'colored, chapter 900 and above'):
    site = cl_chapter_site[i].replace('-', '0') 

    if (int(site[-3:None]) >= 900) or (int(site[-4:None]) >= 900):
        cl_chapters.append(cl_chapter_site[i])
        
for j in tqdm(range(len(bw_chapter_site)), 'grayscale, chapter 900 and above'):
    site = bw_chapter_site[j].replace('-', '0')
    site = site.replace('.5', '')
    site = site.replace('.1', '')

    if (int(site[-3:None]) >= 900) or (int(site[-4:None]) >= 900):
        bw_chapters.append(bw_chapter_site[j])
        
# cl_chapters.sort()
# bw_chapters.sort()

grayscale, chapter 900 and above: 100%|██████████| 1060/1060 [00:00<00:00, 341707.96it/s]


In [5]:
pprint(len(cl_chapters))
pprint(len(bw_chapters))

pprint(cl_chapters[-3:None])
pprint(bw_chapters[-3:None])

89
159
['https://ww9.readonepiece.com/chapter/one-piece-digital-colored-comics-chapter-902',
 'https://ww9.readonepiece.com/chapter/one-piece-digital-colored-comics-chapter-901',
 'https://ww9.readonepiece.com/chapter/one-piece-digital-colored-comics-chapter-900']
['https://ww9.readonepiece.com/chapter/one-piece-chapter-901',
 'https://ww9.readonepiece.com/chapter/one-piece-chapter-900',
 'https://ww9.readonepiece.com/chapter/one-piece-chapter-52-5']


# Parsing all image urls from each chapter

## colored

In [6]:
cl = []
for i in tqdm(range(len(cl_chapters)), "extracting images"):
# for i in tqdm(range(1), "extracting images"):
    cl_imgs = []
    
    html_text = requests.get(cl_chapters[i], 'r').text
    soup = BeautifulSoup(html_text, 'lxml')

    all_imgs = soup.find_all('div', class_ = 'text-center')
    for img in all_imgs:
        site = img.find("img", class_ = 'mb-3 mx-auto js-page')
        if site is not None:
            cl_imgs.append(site['src'])
            
    cl = cl + cl_imgs[1:None]
            
#     pprint(cl_imgs)
#     pprint(cl)
#     raise ValueError("done")

extracting images: 100%|██████████| 89/89 [02:16<00:00,  1.54s/it]


In [7]:
print(len(cl))
print(cl[0])

1461
https://i.imgur.com/mRmn97e.jpeg


## grayscale

In [8]:
bw = []
for i in tqdm(range(len(bw_chapters)), "extracting images"):
# for i in tqdm(range(1), "extracting images"):
    bw_imgs = []
    
    html_text = requests.get(bw_chapters[i], 'r').text
    soup = BeautifulSoup(html_text, 'lxml')

    all_imgs = soup.find_all('div', class_ = 'text-center')
    for img in all_imgs:
        site = img.find("img", class_ = 'mb-3 mx-auto js-page')
        if site is not None:
            bw_imgs.append(site['src'])    
            
    bw = bw + bw_imgs[1:None]

extracting images: 100%|██████████| 159/159 [04:20<00:00,  1.64s/it]


In [9]:
print(len(bw))
print(bw[0])

2679
https://cdn.readonepiece.com/file/mangap/2/11049000/2.jpeg


# Download the images

## colored

In [10]:
pathname = './colored'
# if path doesn't exist, make that path dir
if not os.path.isdir(pathname):
    os.makedirs(pathname)
    
for i in tqdm(range(len(cl)), "downloading images"):
    url = cl[i].replace('\r', '')  
#     url = 'https://cdn.readonepiece.com/file/mangap/3258/10900000/1.jpeg'
    filename = os.path.join(pathname, str(i) + str('.png'))

    response = requests.get(url)
    if response.ok:    
        with open(filename, 'wb') as f:
                for block in response.iter_content(1024):
                    if not block:
                        break

                    f.write(block)
    else:
        print(response)

downloading images: 100%|██████████| 1461/1461 [06:57<00:00,  3.50it/s]


## grayscale

In [11]:
pathname = './grayscale'
# if path doesn't exist, make that path dir
if not os.path.isdir(pathname):
    os.makedirs(pathname)
    
for i in tqdm(range(len(bw)), "downloading images"):
    url = bw[i].replace('\r', '')
    filename = os.path.join(pathname, str(i) + str('.png'))

    response = requests.get(url)
    if response.ok:    
        with open(filename, 'wb') as f:
                for block in response.iter_content(1024):
                    if not block:
                        break

                    f.write(block)
    else:
        print(response)

downloading images: 100%|██████████| 2679/2679 [09:26<00:00,  4.73it/s]
